In [2]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, Q
import pandas as pd
# Create the client instance
# client = Elasticsearch("http://localhost:9200")

# # Successful response!
# client.info()

ELASTIC_PASSWORD = "nqWXecniTzykQWUfZpxPliLq"

# Found in the 'Manage Deployment' page
CLOUD_ID = "Recipes-chefWnos:ZXVyb3BlLXdlc3QzLmdjcC5jbG91ZC5lcy5pbzo0NDMkZmYwOTc5MjM4MjhmNDdhNjk4NTZmYzg2M2IzZGNhYTMkNzMwZmZkOGRjZDgxNDRhYzkwMDI1NzM0OTA0ZTM0ZGQ="

# Create the client instance
client = Elasticsearch(
    cloud_id=CLOUD_ID,
    http_auth=("elastic", ELASTIC_PASSWORD)
)

# Successful response!
client.info()

c:\Users\Fastora\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\Fastora\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\Fastora\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


{'name': 'instance-0000000001',
 'cluster_name': 'ff097923828f47a69856fc863b3dcaa3',
 'cluster_uuid': 'pxX7iuB_TNOB7WKUuYai4A',
 'version': {'number': '8.8.0',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': 'c01029875a091076ed42cdb3a41c10b1a9a5a20f',
  'build_date': '2023-05-23T17:16:07.179039820Z',
  'build_snapshot': False,
  'lucene_version': '9.6.0',
  'minimum_wire_compatibility_version': '7.17.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'You Know, for Search'}

In [3]:
recipes = pd.read_json('data/recipes_cleaned_2.json')

In [4]:
for i, data in recipes.iterrows():
    ing = []
    for j in data['ingredients']:
        ing.append(j['ingredient'])
    data['ingredients'] = ing


In [4]:
recipes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14273 entries, 0 to 14272
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         14273 non-null  object
 1   ingredients  14273 non-null  object
 2   steps        14273 non-null  object
 3   description  14273 non-null  object
 4   time         14273 non-null  object
 5   tags         14273 non-null  object
 6   cuisine      14273 non-null  object
 7   img          10788 non-null  object
dtypes: object(8)
memory usage: 892.2+ KB


In [5]:
def create_index(client, index_name):
    # Define the index settings and mappings
    if client.indices.exists(index_name):
        pass
    index_settings = {
        "settings": {
            "analysis": {
                "filter": {
                    "arabic_stemmer": {
                        "type": "stemmer",
                        "language": "arabic"
                    },
                    "arabic_stop": {
                        "type": "stop",
                        "stopwords": "_arabic_"
                    }
                },
                "analyzer": {
                    "rebuilt_arabic": {
                        "type": "custom",
                        "tokenizer": "standard",
                        "filter": ["lowercase", "decimal_digit", "arabic_stop", "arabic_normalization", "arabic_stemmer"]
                    }
                }
            }
        },
        "mappings": {
            "properties": {
                "Id": {
                    "type": "long"
                },
                "name": {
                    "type": "text",
                    "analyzer": "rebuilt_arabic",
                    "search_analyzer": "rebuilt_arabic"
                },
                "ingredients": {
                    "type": "text",
                    "analyzer": "rebuilt_arabic",
                    "search_analyzer": "rebuilt_arabic"
                },
                "steps": {
                    "type": "text",
                    "analyzer": "rebuilt_arabic",
                    "search_analyzer": "rebuilt_arabic"
                },
                "tags": {
                    "type": "text",
                    "analyzer": "rebuilt_arabic",
                    "search_analyzer": "rebuilt_arabic"
                }
            }
        }
    }

    # Create the index with the specified settings and mappings
    create_index_response = client.indices.create(index=index_name, body=index_settings)
    return create_index_response
# Call the function to create the index
response = create_index(client=client, index_name="recipes")
print(response)

C:\Users\Fastora\AppData\Local\Temp\ipykernel_8900\4178635385.py:3: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  if client.indices.exists(index_name):
C:\Users\Fastora\AppData\Local\Temp\ipykernel_8900\4178635385.py:57: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  create_index_response = client.indices.create(index=index_name, body=index_settings)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'recipes'}


In [6]:
def get_recipe_by_id(client, index_name, id):
    search = Search(using=client, index=index_name).query("match", Id=id)
    response = search.execute()
    if response:
        return response[0].to_dict()
    else:
        return None

In [7]:
get_recipe_by_id(client, "recipes", 1)

In [8]:
df_recipes = recipes
#join ingredients list to one string
df_recipes['ingredients'] = df_recipes['ingredients'].apply(lambda x: ' '.join(x))
#join tags list to one string
df_recipes['tags'] = df_recipes['tags'].apply(lambda x: ' '.join(x))
#join steps list to one string
df_recipes['steps'] = df_recipes['steps'].apply(lambda x: ' '.join(x))


In [5]:
recipes['Id'] = recipes.index


In [8]:
#get recipe of id 8841
recipes.loc[8841]

name                                             مكرونة بالجمبري
ingredients    [الجمبري, معكرونة, البروكولي, زيت الزيتون, ملح...
steps          [حمى الفرن على درجة حرارة 200 مئوية., تبلي الج...
description                                     مكرونة بالجمبري 
time                                                   30 دقيقة 
tags           [المكرونة, مأكولات بحرية, مكرونة, طبخات الجمبر...
cuisine                                                    غربي 
img            https://kitchen.sayidaty.net/uploads/small/17/...
Id                                                          8841
Name: 8841, dtype: object

In [10]:
def add_recipes(client, index, recipes):
    #add to recipes id column from its index
    recipes['Id'] = recipes.index
    # Convert the dataframe to a dictionary
    recipes_dict = recipes.to_dict("records")
    for recipe in recipes_dict:
        # Index the dictionary of each recipe into Elasticsearch
        resp = client.index(index=index, document=recipe)
        if resp['result'] != 'created':
            print(resp)


In [12]:
add_recipes(client, 'recipes', df_recipes)

In [11]:
def search_by_ingredients(ingredients_query, client, index):
    # Define the search query
    search_query = Q(
        "multi_match",
        fields=["ingredients^2", "name^2", "steps"],
        query=ingredients_query,
        fuzziness="AUTO"
    )

    # Create a search object
    search = client.search(index=index, body={"query": search_query.to_dict()})

    ids = []
    for hit in search["hits"]["hits"]:
        ids.append(hit['_source']['Id'])
        
    return search["hits"]["hits"], ids

In [13]:
q = "بطاطس رز بصل"
result = search_by_ingredients(q, client, 'recipes')
print(result)

C:\Users\Fastora\AppData\Local\Temp\ipykernel_8900\3548199306.py:11: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  search = client.search(index=index, body={"query": search_query.to_dict()})


([{'_index': 'recipes', '_id': 'jwDvh4gBtjLtA0NXU84u', '_score': 18.75136, '_source': {'name': 'الرز الأصفر بالبصل..', 'ingredients': 'الأرز مرق دجاج ملح البصل الكركم الزيت النباتي الفلفل الأخضر بهارات', 'steps': '1. يسكب الزيت في قدر توضع على نار هادئة، ويضاف له البصل المفروم، ويقلب لدقيقتين، فالأرز الذي يقلب أيضاً لدقيقتين، ثم ينثر الكركم والملح ويوضع مرق الدجاج والبهار. 2. يقلب هذا المزيج، وتغطى القدر، مع التقليب كلّ 5 دقائق حتى يجف الماء تماماً وينضج الأرز. 3. ثم، يضاف بعض الزيت في قدر نظيفة، ويقلى الفلفل الأخضر الحلو حتى يذبل، ثم يخلط بالأرز. 4. \xa0يمكن تزيين هذا الطبق بأوراق البصل الأخصر المقطعة.', 'description': '\n                        ', 'time': '0 دقيقة ', 'tags': 'أطباق الأرز وصفات الأرز أطباق رمضانية', 'cuisine': ' عربي ', 'img': None, 'Id': 3803}}, {'_index': 'recipes', '_id': 'BgDth4gBtjLtA0NXE8oX', '_score': 15.860619, '_source': {'name': 'بطاطس مقلية مع البصل', 'ingredients': 'البطاطس البصل الزبدة زيت الزيتون بودرة الثوم أعشاب مجففة ملح فلفل أسود', 'steps': '1.\xa0 س

In [14]:
ids = result[1]

In [15]:
for i in ids:
    print(df_recipes.loc[df_recipes['Id'] == int(i)]['name'].values[0])

الرز الأصفر بالبصل..
بطاطس مقلية مع البصل
بصل مشوي بالبطاطس
شوربة البطاطس بالبصل
بطاطس حلقات بالبصل
الرز البخاري الأصلي
رز باللحمة والبطاطس والعدس
بطاطس بالتوم والبصل الأخضر
شوربة البطاطس والبصل الأخضر
دجاج مشوي بالبطاطس والبصل


In [16]:
def search_by_tags(tags_query, client, index):
    # Define the search query
    search_query = Q(
        "multi_match",
        fields=["tags^2", "name", "description"],
        query=tags_query,
        fuzziness="AUTO"
    )

    # Create a search object
    search = client.search(index=index, body={"query": search_query.to_dict()})

    ids = []
    for hit in search["hits"]["hits"]:
        ids.append(hit['_source']['Id'])
        
    return search["hits"]["hits"], ids

In [17]:
def search_by_name(name_query, client, index):
    # Define the search query
    search_query = Q(
        "multi_match",
        fields=["name^2", "ingredients", "steps"],
        query=name_query,
        fuzziness="AUTO"
    )

    # Create a search object
    search = client.search(index=index, body={"query": search_query.to_dict()})

    ids = []
    for hit in search["hits"]["hits"]:
        ids.append(hit['_source']['Id'])
        
    return search["hits"]["hits"], ids

In [18]:
result = search_by_name("كباب في القلاية الهوائية", client, 'recipes')
print(result)


C:\Users\Fastora\AppData\Local\Temp\ipykernel_8900\3468369990.py:11: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  search = client.search(index=index, body={"query": search_query.to_dict()})


([{'_index': 'recipes', '_id': 'tQDph4gBtjLtA0NXBb9v', '_score': 36.210842, '_source': {'name': 'كباب في القلاية الهوائية', 'ingredients': 'لحم بقر مفروم لحم غنم بقدونس سماق ملح فلفل أسود البصل بقسماط السمنة', 'steps': 'في وعاء ضعي لحم البقر، ولحم الغنم، والسمنة، والبصل، والبقسماط، والبقدونس، ثم نكهي بالملح، والفلفل الأسود، والسماق. قلبي المكونات حتى تتداخل وشكلي خليط الكباب إلى أصابع. أدخلي أصابع الكباب إلى القلاية الهوائية لحوالي 10 دقائق على حرارة 180 درجة مئوية. أخرجي أصابع الكباب من القلاية الهوائية وقدميها ساخنة.', 'description': 'كباب في القلاية الهوائية ... تتعدد مميزات وإيجابيات استخدامك للقلاية الهوائية في تحضير الأطعمة، ومنها أنها تحافظ على مطبخك خالٍ من روائح القلي، وتوفر الوقت والجهد، وتساعدكِ في تحضير الأكلات بكمية زيت قليلة جداً، جربي تحضير وصفة من ', 'time': '60 دقيقة ', 'tags': 'الكباب طبخات اللحم المفروم لحم مفروم وصفات الكباب وصفات سريعة بقدونس طبخات سهلة اكلات سهله وصفات سهلة وسريعة', 'cuisine': ' عربي ', 'img': 'https://kitchen.sayidaty.net/uploads/small/74/7489555

In [19]:
result = search_by_tags('كباب في القلاية الهوائية', client, 'recipes')
print(result)

([{'_index': 'recipes', '_id': 'tQDph4gBtjLtA0NXBb9v', '_score': 18.105421, '_source': {'name': 'كباب في القلاية الهوائية', 'ingredients': 'لحم بقر مفروم لحم غنم بقدونس سماق ملح فلفل أسود البصل بقسماط السمنة', 'steps': 'في وعاء ضعي لحم البقر، ولحم الغنم، والسمنة، والبصل، والبقسماط، والبقدونس، ثم نكهي بالملح، والفلفل الأسود، والسماق. قلبي المكونات حتى تتداخل وشكلي خليط الكباب إلى أصابع. أدخلي أصابع الكباب إلى القلاية الهوائية لحوالي 10 دقائق على حرارة 180 درجة مئوية. أخرجي أصابع الكباب من القلاية الهوائية وقدميها ساخنة.', 'description': 'كباب في القلاية الهوائية ... تتعدد مميزات وإيجابيات استخدامك للقلاية الهوائية في تحضير الأطعمة، ومنها أنها تحافظ على مطبخك خالٍ من روائح القلي، وتوفر الوقت والجهد، وتساعدكِ في تحضير الأكلات بكمية زيت قليلة جداً، جربي تحضير وصفة من ', 'time': '60 دقيقة ', 'tags': 'الكباب طبخات اللحم المفروم لحم مفروم وصفات الكباب وصفات سريعة بقدونس طبخات سهلة اكلات سهله وصفات سهلة وسريعة', 'cuisine': ' عربي ', 'img': 'https://kitchen.sayidaty.net/uploads/small/74/7489555

C:\Users\Fastora\AppData\Local\Temp\ipykernel_8900\2433964035.py:11: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  search = client.search(index=index, body={"query": search_query.to_dict()})
